In [1]:
import pandas as pd

df = pd.read_csv("Dataset.csv")  # Use your actual CSV file path
print(df.head())
print(df.info())


   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

In [2]:
df.dropna(inplace=True)


In [3]:
df['Cuisines'] = df['Cuisines'].str.lower()
df['City'] = df['City'].str.lower()


In [4]:
def combine_features(row):
    return f"{row['Cuisines']} price{row['Price range']}"

df['combined'] = df.apply(combine_features, axis=1)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])


In [6]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 36614 stored elements and shape (9542, 154)>

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tfidf_matrix)


In [8]:
user_cuisine = "japanese"
user_price_range = 3


In [9]:
user_pref = f"{user_cuisine.lower()} price{user_price_range}"
user_vec = vectorizer.transform([user_pref])

similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()
top_indices = similarities.argsort()[-5:][::-1]  # Top 5 matches

recommendations = df.iloc[top_indices][['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']]
print(recommendations)


     Restaurant Name  Cuisines  Price range  Aggregate rating
4335          Tamura  japanese            3               3.3
1517        Daikichi  japanese            3               3.6
3809          Tamura  japanese            3               3.5
2171           Tokyo  japanese            3               3.0
1466         Kuuraku  japanese            3               3.9
